In [1]:
import numpy as np
import xarray as xr
import xgcm
import zarr

In [2]:
xr.__version__

'0.10.7'

In [3]:
import dask

In [4]:
dask.__version__

'0.18.1+38.gca8d62e9'

In [5]:
# Define face_connections for grid object
face_connections = {'face':
                    {0: {'X':  ((12, 'Y', False), (3, 'X', False)),
                         'Y':  (None,             (1, 'Y', False))},
                     1: {'X':  ((11, 'Y', False), (4, 'X', False)),
                         'Y':  ((0, 'Y', False),  (2, 'Y', False))},
                     2: {'X':  ((10, 'Y', False), (5, 'X', False)),
                         'Y':  ((1, 'Y', False),  (6, 'X', False))},
                     3: {'X':  ((0, 'X', False),  (9, 'Y', False)),
                         'Y':  (None,             (4, 'Y', False))},
                     4: {'X':  ((1, 'X', False),  (8, 'Y', False)),
                         'Y':  ((3, 'Y', False),  (5, 'Y', False))},
                     5: {'X':  ((2, 'X', False),  (7, 'Y', False)),
                         'Y':  ((4, 'Y', False),  (6, 'Y', False))},
                     6: {'X':  ((2, 'Y', False),  (7, 'X', False)),
                         'Y':  ((5, 'Y', False),  (10, 'X', False))},
                     7: {'X':  ((6, 'X', False),  (8, 'X', False)),
                         'Y':  ((5, 'X', False),  (10, 'Y', False))},
                     8: {'X':  ((7, 'X', False),  (9, 'X', False)),
                         'Y':  ((4, 'X', False),  (11, 'Y', False))},
                     9: {'X':  ((8, 'X', False),  None),
                         'Y':  ((3, 'X', False),  (12, 'Y', False))},
                     10: {'X': ((6, 'Y', False),  (11, 'X', False)),
                          'Y': ((7, 'Y', False),  (2, 'X', False))},
                     11: {'X': ((10, 'X', False), (12, 'X', False)),
                          'Y': ((8, 'Y', False),  (1, 'X', False))},
                     12: {'X': ((11, 'X', False), None),
                          'Y': ((9, 'Y', False),  (0, 'X', False))}}}

### Load dataset

In [6]:
# Main disagnostic output
ds_main = xr.open_zarr('/rigel/ocp/users/jt2796/eccov4r3_output')
coords_main = ds_main.coords.to_dataset().reset_coords()
ds_main = ds_main.reset_coords(drop=True)

# Budget terms
ds_budg = xr.open_zarr('/rigel/ocp/users/jt2796/eccov4r3_budgets')
#ds_budg.time.values = ds_main.time.values
coords_budg = ds_budg.coords.to_dataset().reset_coords()
ds_budg = ds_budg.reset_coords(drop=True)

### Volume

In [7]:
# Cell z size
drF = coords_main.drF
rA = coords_main.rA
hFacC = coords_main.hFacC

# Volume (m^3)
vol = (rA*drF*hFacC).transpose('face','k','j','i')

In [8]:
#### Seperate averages and snapshots
ds_ave = ds_main[['ETAN','THETA','SALT','UVELMASS','VVELMASS','WVELMASS',
                  'ADVx_TH','ADVx_SLT','ADVy_TH','ADVy_SLT','ADVr_TH','ADVr_SLT']]

ds_snp = ds_main[['ETAN_snp','THETA_snp','SALT_snp']].rename({'time_snp':'time'})

In [9]:
# Remove oceFWflx from WVELMASS
WVELMASS = ds_main.WVELMASS.transpose('time','face','k_l','j','i')
oceFWflx = ds_main.oceFWflx.assign_coords(k_l=0).expand_dims('k_l').transpose('time','face','k_l','j','i')

rhoconst = 1029
oceFWflx = (oceFWflx/rhoconst)
WVELMASS = xr.concat([WVELMASS.sel(k_l=0) + oceFWflx, WVELMASS[:,:,1:]], 
                     dim='k_l').transpose('time','face','k_l','j','i')

In [10]:
#### Monthly means
ds_ave_clim = ds_ave.groupby('time.month').mean('time')
WVELMASS_clim = WVELMASS.groupby('time.month').mean('time')
ds_snp_clim = ds_snp.groupby('time.month').mean('time')
ds_budg_clim = ds_budg.groupby('time.month').mean('time')

#### Monthly anomalies
ds_ave_anom = ds_ave.groupby('time.month') - ds_ave_clim
WVELMASS_anom = WVELMASS.groupby('time.month') - WVELMASS_clim
ds_snp_anom = ds_snp.groupby('time.month') - ds_snp_clim
ds_budg_anom = ds_budg.groupby('time.month') - ds_budg_clim

In [11]:
#### Monthly mean terms
grid = xgcm.Grid(ds_ave_clim, face_connections=face_connections)

# Transport
u_clim = (ds_ave_clim.UVELMASS * coords_main.dyG * coords_main.drF).transpose('month','face','k','j','i_g')
v_clim = (ds_ave_clim.VVELMASS * coords_main.dxG * coords_main.drF).transpose('month','face','k','j_g','i')
w_clim = (WVELMASS_clim * coords_main.rA).transpose('month','face','k_l','j','i')

# Potential Temperature (degC)
THETA_clim = ds_ave_clim.THETA.transpose('month','face','k','j','i')
THETA_clim_at_u = grid.interp(THETA_clim, 'X', boundary='extend')
THETA_clim_at_v = grid.interp(THETA_clim, 'Y', boundary='extend')
THETA_clim_at_w = grid.interp(THETA_clim, 'Z', boundary='extend')

In [12]:
#### Monthly anomaly terms
grid = xgcm.Grid(ds_ave_anom, face_connections=face_connections)

# Transport
u_anom = (ds_ave_anom.UVELMASS * coords_main.dyG * coords_main.drF).transpose('time','face','k','j','i_g')
v_anom = (ds_ave_anom.VVELMASS * coords_main.dxG * coords_main.drF).transpose('time','face','k','j_g','i')
w_anom = (WVELMASS_anom * coords_main.rA).transpose('time','face','k_l','j','i')

# Temperature
THETA_anom = ds_ave_anom.THETA.transpose('time','face','k','j','i')
THETA_anom_at_u = grid.interp(THETA_anom, 'X', boundary='extend')
THETA_anom_at_v = grid.interp(THETA_anom, 'Y', boundary='extend')
THETA_anom_at_w = grid.interp(THETA_anom, 'Z', boundary='extend')

### Monthly anomaly budget
$$\frac{\partial\theta^{\prime}}{\partial t} + \overline{\bf{u}}^m\cdot\nabla\theta^{\prime} + \bf{u}^{\prime} \cdot \overline{\nabla\theta}^m -\nabla \cdot ({\bf{u}}^{\prime}\,\theta^{\prime}-\overline{\bf{u}^{\prime}\,\theta^{\prime}}^m) = -\nabla \cdot {\bf{F_{diff}}}^{\prime} - F_{\textrm{forc}}^{\prime}$$

In [13]:
#### Anomaleous heat tendency
tendH_anom = ds_budg_anom.tendH

#### Anomaleous forcing
forcH_anom = ds_budg_anom.forcH

#### Anomaleous diffusive heat convergence
dif_hConvH = ds_budg_anom.dif_hConvH
dif_vConvH = ds_budg_anom.dif_vConvH
dif_ConvH_anom = dif_hConvH + dif_vConvH

In [14]:
#### Mean advection of anomaleous temperature
uclimTanom = u_clim * THETA_anom_at_u.groupby('time.month')
vclimTanom = v_clim * THETA_anom_at_v.groupby('time.month')
wclimTanom = w_clim * THETA_anom_at_w.groupby('time.month')

# Convergence
ADVxy_diff = grid.diff_2d_vector({'X' : uclimTanom, 'Y' : vclimTanom}, boundary = 'fill')
ADVx_diffx = ADVxy_diff['X']
ADVy_diffy = ADVxy_diff['Y']
adv_hConvH = (-(ADVx_diffx + ADVy_diffy)/vol)
adv_vConvH = (grid.diff(wclimTanom, 'Z', boundary='fill')/vol)

adv_ConvH_Uclim_Tanom = adv_hConvH + adv_vConvH

In [15]:
#### Anomaleous advection of mean temperature
uanomTclim = u_anom.groupby('time.month') * THETA_clim_at_u
vanomTclim = v_anom.groupby('time.month') * THETA_clim_at_v
wanomTclim = w_anom.groupby('time.month') * THETA_clim_at_w

# Convergence
ADVxy_diff = grid.diff_2d_vector({'X' : uanomTclim, 'Y' : vanomTclim}, boundary = 'fill')
ADVx_diffx = ADVxy_diff['X']
ADVy_diffy = ADVxy_diff['Y']
adv_hConvH = (-(ADVx_diffx + ADVy_diffy)/vol)
adv_vConvH = (grid.diff(wanomTclim, 'Z', boundary='fill')/vol)

adv_ConvH_Uanom_Tclim = adv_hConvH + adv_vConvH

/rigel/home/jt2796/dask/dask/array/core.py:3460: PerformanceWarning: Increasing number of chunks by factor of 24
  **atop_kwargs)
/rigel/home/jt2796/dask/dask/array/core.py:3460: PerformanceWarning: Increasing number of chunks by factor of 24
  **atop_kwargs)
/rigel/home/jt2796/dask/dask/array/core.py:3460: PerformanceWarning: Increasing number of chunks by factor of 24
  **atop_kwargs)
/rigel/home/jt2796/dask/dask/array/core.py:3460: PerformanceWarning: Increasing number of chunks by factor of 24
  **atop_kwargs)
/rigel/home/jt2796/dask/dask/array/core.py:3460: PerformanceWarning: Increasing number of chunks by factor of 24
  **atop_kwargs)
/rigel/home/jt2796/dask/dask/array/core.py:3460: PerformanceWarning: Increasing number of chunks by factor of 24
  **atop_kwargs)
/rigel/home/jt2796/dask/dask/array/core.py:3460: PerformanceWarning: Increasing number of chunks by factor of 24
  **atop_kwargs)
/rigel/home/jt2796/dask/dask/array/core.py:3460: PerformanceWarning: Increasing number of 

In [16]:
#### Anomaleous advection of anomaleous temperature
uanomTanom = u_anom * THETA_anom_at_u
vanomTanom = v_anom * THETA_anom_at_v
wanomTanom = w_anom * THETA_anom_at_w

uanomTanom_clim = uanomTanom.groupby('time.month').mean('time')
vanomTanom_clim = vanomTanom.groupby('time.month').mean('time')
wanomTanom_clim = wanomTanom.groupby('time.month').mean('time')

uanomTanom_clim_anom = uanomTanom.groupby('time.month') - uanomTanom_clim
vanomTanom_clim_anom = vanomTanom.groupby('time.month') - vanomTanom_clim
wanomTanom_clim_anom = wanomTanom.groupby('time.month') - wanomTanom_clim

# Convergence
ADVxy_diff = grid.diff_2d_vector({'X' : uanomTanom_clim_anom, 'Y' : vanomTanom_clim_anom}, boundary = 'fill')
ADVx_diffx = ADVxy_diff['X']
ADVy_diffy = ADVxy_diff['Y']
adv_hConvH = (-(ADVx_diffx + ADVy_diffy)/vol)
adv_vConvH = (grid.diff(wanomTanom_clim_anom, 'Z', boundary='fill')/vol)

adv_ConvH_Uanom_Tanom = adv_hConvH + adv_vConvH

In [17]:
#### Residual in the advective flux

# Transport
u_transport = (ds_ave.UVELMASS * coords_main.dyG * coords_main.drF)
v_transport = (ds_ave.VVELMASS * coords_main.dxG * coords_main.drF)
w_transport = (WVELMASS * rA)

# Temperature
THETA = ds_main.THETA
THETA_at_u = grid.interp(THETA, 'X', boundary='extend')
THETA_at_v = grid.interp(THETA, 'Y', boundary='extend')
THETA_at_w = grid.interp(THETA, 'Z', boundary='extend')

# Advection
uT = (u_transport * THETA_at_u)
vT = (v_transport * THETA_at_v)
wT = (w_transport * THETA_at_w)

# Convergence
ADVxy_diff = grid.diff_2d_vector({'X' : uT, 'Y' : vT}, boundary = 'fill')
ADVx_diffx = ADVxy_diff['X']
ADVy_diffy = ADVxy_diff['Y']
adv_hConvH = (-(ADVx_diffx + ADVy_diffy)/vol).transpose('time','face','k','j','i')
adv_vConvH = (grid.diff(wT, 'Z', boundary='fill')/vol).transpose('time','face','k','j','i')

# Reconstructed
adv_ConvH_reco = adv_hConvH + adv_vConvH

# True (diagnostic)
adv_ConvH_true = ds_budg.adv_hConvH + ds_budg.adv_vConvH

# Residual
adv_ConvH_res = adv_ConvH_true - adv_ConvH_reco
adv_ConvH_res_clim = adv_ConvH_res.groupby('time.month').mean('time')
adv_ConvH_res_anom = adv_ConvH_res.groupby('time.month') - adv_ConvH_res_clim

### Save to dataset

In [18]:
ds = xr.Dataset(data_vars={})
ds['tendH_anom'] = tendH_anom
ds['forcH_anom'] = forcH_anom
ds['dif_ConvH_anom'] = dif_ConvH_anom
ds['adv_ConvH_Uclim_Tanom'] = adv_ConvH_Uclim_Tanom
ds['adv_ConvH_Uanom_Tclim'] = adv_ConvH_Uanom_Tclim
ds['adv_ConvH_Uanom_Tanom'] = adv_ConvH_Uanom_Tanom
ds['adv_ConvH_res_anom'] = adv_ConvH_res_anom

$$\frac{\partial\theta^{\prime}}{\partial t} + \overline{\bf{u}}^m\cdot\nabla\theta^{\prime} + \bf{u}^{\prime} \cdot \overline{\nabla\theta}^m -\nabla \cdot ({\bf{u}}^{\prime}\,\theta^{\prime}-\overline{\bf{u}^{\prime}\,\theta^{\prime}}^m) = -\nabla \cdot {\bf{F_{diff}}}^{\prime} - F_{\textrm{forc}}^{\prime}$$
- `tendH_anom`: Anomaleous tendency ($\frac{\partial\theta^{\prime}}{\partial t}$)
- `forcH_anom`: Anomaleous forcing ($F_{\textrm{forc}}^{\prime}$)
- `dif_ConvH_anom`: Anomaleous diffusion ($\nabla \cdot {\bf{F_{diff}}}^{\prime}$)
- `adv_ConvH_Uclim_Tanom`: Mean advection of T anomalies ($\overline{\bf{u}}^m\cdot\nabla\theta^{\prime}$)
- `adv_ConvH_Uanom_Tclim`: Anomaleous advection of mean T ($\bf{u}^{\prime} \cdot \overline{\nabla\theta}^m$)
- `adv_ConvH_Uanom_Tanom`: $\nabla \cdot ({\bf{u}}^{\prime}\,\theta^{\prime}-\overline{\bf{u}^{\prime}\,\theta^{\prime}}^m)$
- `adv_ConvH_res_anom`: Residual in advection